In [1]:
import sys
sys.path.append("../src")

In [2]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher

from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy

from functools import reduce
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import statsmodels.api as sm

import urllib.parse
import requests
import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import matplotlib.pylab as pylab
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)
sns.set(style="whitegrid", palette="dark")
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Project Bond King\Curvy-CUSIPs\notebooks\../.env


In [3]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

In [163]:
as_of_date = datetime(2024, 11, 6)
sdr = curve_data_fetcher.dtcc_sdr_fetcher.fetch_dtcc_sdr_data_timeseries(
    start_date=as_of_date , end_date=as_of_date, agency="CFTC", asset_class="RATES"
)
sdr_df = sdr[as_of_date]

FETCHING DTCC SDR DATASETS...: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s]


In [164]:
swaption_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_swaption_upis.csv")
capfloor_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_capfloor_upis.csv")
fixed_float_swaps_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_fixed_float_swaps_upis.csv")

In [165]:
swaption_underlying_conds = (
    # USD SOFR OIS SWAP
    (fixed_float_swaps_dsb_lookup_df["Header_UseCase"] == "Fixed_Float_OIS") &
    (fixed_float_swaps_dsb_lookup_df["Derived_UnderlyingAssetType"] == "Overnight Index Swap (OIS)") &
    (fixed_float_swaps_dsb_lookup_df["Derived_ShortName"] == "NA/Swap OIS USD") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_NotionalCurrency"] == "USD") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_NotionalSchedule"] == "Constant") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_DeliveryType"] == "PHYS") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_ReferenceRateTermValue"] == 1) &
    (fixed_float_swaps_dsb_lookup_df["Attributes_ReferenceRateTermUnit"] == "DAYS") &
    (fixed_float_swaps_dsb_lookup_df["Derived_UnderlierName"] == "USD-SOFR-OIS Compound")   
)

underlying_row = fixed_float_swaps_dsb_lookup_df[swaption_underlying_conds]
assert len(underlying_row) == 1
underlying_dict = underlying_row.to_dict("records")[0]
print("Underlying: ")
display(underlying_dict)

swaption_conds = (
    (swaption_dsb_lookup_df["Attributes_UnderlyingInstrumentUPI"] == underlying_dict["Identifier_UPI"]) &
    (swaption_dsb_lookup_df["Attributes_ValuationMethodorTrigger"] == "Vanilla") &
    (swaption_dsb_lookup_df["Attributes_DeliveryType"] == "PHYS")
)
receiver_conds = (
    swaption_conds & (swaption_dsb_lookup_df["Derived_CFIOptionStyleandType"] == "European-Put")
)
payer_conds = (
    swaption_conds & (swaption_dsb_lookup_df["Derived_CFIOptionStyleandType"] == "European-Call")
)

receiver_swpation_row = swaption_dsb_lookup_df[receiver_conds]
assert len(receiver_swpation_row) == 1
receiver_swpation_dict = receiver_swpation_row.to_dict("records")[0]
print("Receiver: ")
display(receiver_swpation_dict)

payer_swpation_row = swaption_dsb_lookup_df[payer_conds]
assert len(payer_swpation_row) == 1
payer_swpation_dict = payer_swpation_row.to_dict("records")[0]
print("Payer: ")
display(payer_swpation_dict)

Underlying: 


{'TemplateVersion': 1,
 'Header_AssetClass': 'Rates',
 'Header_InstrumentType': 'Swap',
 'Header_UseCase': 'Fixed_Float_OIS',
 'Header_Level': 'UPI',
 'Identifier_UPI': 'QZPB5VSBGRCD',
 'Identifier_Status': 'New',
 'Identifier_StatusReason': nan,
 'Identifier_LastUpdateDateTime': '2023-10-15T13:20:35',
 'Derived_ClassificationType': 'SRHCSP',
 'Derived_UnderlyingAssetType': 'Overnight Index Swap (OIS)',
 'Derived_SingleorMultiCurrency': 'Single Currency',
 'Derived_CFIDeliveryType': 'Physical',
 'Derived_ShortName': 'NA/Swap OIS USD',
 'Derived_UnderlierName': 'USD-SOFR-OIS Compound',
 'Attributes_NotionalCurrency': 'USD',
 'Attributes_ReferenceRate': 'USD-SOFR-OIS Compound',
 'Attributes_ReferenceRateTermValue': 1,
 'Attributes_ReferenceRateTermUnit': 'DAYS',
 'Attributes_NotionalSchedule': 'Constant',
 'Attributes_DeliveryType': 'PHYS'}

Receiver: 


{'TemplateVersion': 1,
 'Header_AssetClass': 'Rates',
 'Header_InstrumentType': 'Option',
 'Header_UseCase': 'Swaption',
 'Header_Level': 'UPI',
 'Identifier_UPI': 'QZNLQ8T0N0SX',
 'Identifier_Status': 'New',
 'Identifier_StatusReason': nan,
 'Identifier_LastUpdateDateTime': '2023-11-06T13:17:04',
 'Derived_ClassificationType': 'HRHDVP',
 'Derived_ShortName': 'NA/O P Epn OIS USD',
 'Derived_UnderlierName': 'NA/Swap OIS USD',
 'Derived_UnderlyingAssetType': 'Overnight Index Swap (OIS)',
 'Derived_CFIOptionStyleandType': 'European-Put',
 'Derived_CFIDeliveryType': 'Physical',
 'Attributes_UnderlyingInstrumentUPI': 'QZPB5VSBGRCD',
 'Attributes_NotionalCurrency': 'USD',
 'Attributes_OptionType': 'PUTO',
 'Attributes_OptionExerciseStyle': 'EURO',
 'Attributes_ValuationMethodorTrigger': 'Vanilla',
 'Attributes_DeliveryType': 'PHYS'}

Payer: 


{'TemplateVersion': 1,
 'Header_AssetClass': 'Rates',
 'Header_InstrumentType': 'Option',
 'Header_UseCase': 'Swaption',
 'Header_Level': 'UPI',
 'Identifier_UPI': 'QZZGWPNBF5R3',
 'Identifier_Status': 'New',
 'Identifier_StatusReason': nan,
 'Identifier_LastUpdateDateTime': '2023-11-06T13:17:02',
 'Derived_ClassificationType': 'HRHAVP',
 'Derived_ShortName': 'NA/O Call Epn OIS USD',
 'Derived_UnderlierName': 'NA/Swap OIS USD',
 'Derived_UnderlyingAssetType': 'Overnight Index Swap (OIS)',
 'Derived_CFIOptionStyleandType': 'European-Call',
 'Derived_CFIDeliveryType': 'Physical',
 'Attributes_UnderlyingInstrumentUPI': 'QZPB5VSBGRCD',
 'Attributes_NotionalCurrency': 'USD',
 'Attributes_OptionType': 'CALL',
 'Attributes_OptionExerciseStyle': 'EURO',
 'Attributes_ValuationMethodorTrigger': 'Vanilla',
 'Attributes_DeliveryType': 'PHYS'}

In [201]:
swaption_cols = [
    "Event timestamp",
    "Execution Timestamp",
    "Effective Date",
    "Expiration Date",
    "Maturity date of the underlier",
    "Notional amount-Leg 1",
    "First exercise date",
    "Fixed rate-Leg 1",
    "Option Premium Amount",
    "Strike Price",
    "Unique Product Identifier",
    "UPI FISN",
    "UPI Underlier Name",
]


def format_swaption_time_and_sales(df: pd.DataFrame):
    date_columns = [
        "Event timestamp",
        "Execution Timestamp",
        "Effective Date",
        "Expiration Date",
        "Maturity date of the underlier",
        "First exercise date",
    ]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])
        df[col] = df[col].dt.tz_localize(None)

    df = df[df["Event timestamp"].dt.date >= as_of_date.date()]
    df = df[df["Execution Timestamp"].dt.date >= as_of_date.date()]

    year_day_count = 360
    df["Expiry (d)"] = (df["Expiration Date"] - df["Effective Date"]).dt.days
    df["Expiry (m)"] = df["Expiry (d)"] / 30
    df["Expiry (yr)"] = df["Expiry (d)"] / year_day_count
    df["Swap Tenor"] = (df["Maturity date of the underlier"] - df["Expiration Date"]).dt.days / year_day_count

    df["Notional amount-Leg 1"] = pd.to_numeric(df["Notional amount-Leg 1"].str.replace(",", ""), errors="coerce")
    df["Fixed rate-Leg 1"] = pd.to_numeric(df["Fixed rate-Leg 1"], errors="coerce")
    df["Option Premium Amount"] = pd.to_numeric(df["Option Premium Amount"].str.replace(",", ""), errors="coerce")
    df["Strike Price"] = pd.to_numeric(df["Strike Price"], errors="coerce")
    df = df.dropna(subset=["Notional amount-Leg 1", "Option Premium Amount", "Strike Price", "Maturity date of the underlier", "Fixed rate-Leg 1"])
    df = df[df["Option Premium Amount"] != 0]

    def format_tenor(row: pd.Series):
        if row["Expiry (yr)"] >= 1:
            return f"{int(row['Expiry (yr)'])}y{int(row['Swap Tenor'])}y"
        elif row["Expiry (m)"] >= 1:
            return f"{int(row['Expiry (m)'])}m{int(row['Swap Tenor'])}y"
        else:
            return f"{int(row['Expiry (d)'])}d{int(row['Swap Tenor'])}y"

    df["Tenor"] = df.apply(format_tenor, axis=1)
    df["Option Premium Amount / $mm Notational"] = df["Option Premium Amount"] / (df["Notional amount-Leg 1"] / 1_000_000)
    # df["Forward"] = ((df["Effective Date"] - as_of_date).dt.days / year_day_count).round()
    df = df.drop(columns=["Expiry (d)", "Expiry (m)", "Expiry (yr)", "Swap Tenor"])

    return df


def format_swaption_premium_ohlc(df: pd.DataFrame, normalize_premium=True):
    df = df.sort_values(by=["Tenor", "Strike Price", "Event timestamp"])
    if normalize_premium:
        df["Premium"] = df["Option Premium Amount"] / df["Notional amount-Leg 1"]
    else:
        df["Premium"] = df["Option Premium Amount"]

    ohlc_df = (
        df.groupby(["Tenor", "Strike Price"])
        .agg(
            Open=("Premium", "first"),
            High=("Premium", "max"),
            Low=("Premium", "min"),
            Close=("Premium", "last"),
        )
        .reset_index()
    )
    return ohlc_df

In [202]:
receiver_swaption_df: pd.DataFrame = format_swaption_time_and_sales(
    sdr_df[sdr_df["Unique Product Identifier"] == receiver_swpation_dict["Identifier_UPI"]][swaption_cols]
)
receiver_swaption_df

,Event timestamp,Execution Timestamp,Effective Date,Expiration Date,Maturity date of the underlier,Notional amount-Leg 1,First exercise date,Fixed rate-Leg 1,Option Premium Amount,Strike Price,Unique Product Identifier,UPI FISN,UPI Underlier Name,Tenor,Option Premium Amount / $mm Notational
2763,2024-11-06 22:08:37,2024-11-06 22:08:37,2024-11-06,2026-11-06,2027-11-10,500000000.0,2026-11-06,0.038525,3122125.0,0.038525,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,2y1y,6244.25
2765,2024-11-06 22:08:37,2024-11-06 22:08:37,2024-11-06,2026-11-06,2027-11-10,500000000.0,2026-11-06,0.038525,3122125.0,0.038525,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,2y1y,6244.25
2812,2024-11-06 21:48:23,2024-11-06 21:48:23,2024-11-06,2024-12-06,2054-12-10,100000000.0,2024-12-06,0.037660,2250000.0,0.037660,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,1m30y,22500.00
2983,2024-11-06 20:14:36,2024-11-06 20:14:36,2024-11-06,2025-02-06,2055-02-10,100000000.0,2025-02-06,0.037680,3485000.0,0.037680,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,3m30y,34850.00
2997,2024-11-06 20:14:36,2024-11-06 20:14:36,2024-11-06,2025-02-06,2055-02-10,100000000.0,2025-02-06,0.037680,3485000.0,0.037680,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,3m30y,34850.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17038,2024-11-06 11:55:18,2024-11-06 11:55:18,2024-11-06,2024-12-06,2034-12-10,25000000.0,2024-12-06,0.039350,562500.0,0.039350,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00
17040,2024-11-06 11:55:18,2024-11-06 11:55:18,2024-11-06,2024-12-06,2034-12-10,25000000.0,2024-12-06,0.039350,562500.0,0.039350,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00
17310,2024-11-06 11:55:18,2024-11-06 11:55:18,2024-11-06,2024-12-06,2034-12-10,25000000.0,2024-12-06,0.039350,562500.0,0.039350,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00
17438,2024-11-06 11:32:10,2024-11-06 11:32:10,2024-11-06,2024-12-06,2034-12-10,50000000.0,2024-12-06,0.039350,1125000.0,0.039350,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00


In [203]:
format_swaption_premium_ohlc(receiver_swaption_df)

,Tenor,Strike Price,Open,High,Low,Close
0,10y10y,0.040050,0.175900,0.175900,0.175900,0.175900
1,10y30y,0.034040,0.360800,0.360800,0.360800,0.360800
2,13d10y,0.039000,0.014400,0.014400,0.014400,0.014400
3,15y10y,0.036270,0.196600,0.196600,0.196600,0.196600
4,15y10y,0.036460,0.196400,0.196400,0.196400,0.196400
...,...,...,...,...,...,...
71,6m30y,0.037750,0.096900,0.096900,0.096900,0.096900
72,6m5y,0.038620,0.028100,0.028100,0.028100,0.028100
73,6m5y,0.038650,0.028100,0.028100,0.028100,0.028100
74,7m5y,0.035000,0.002538,0.002538,0.002538,0.002538


In [190]:
payer_swaption_df: pd.DataFrame = format_swaption_time_and_sales(
    sdr_df[sdr_df["Unique Product Identifier"] == payer_swpation_dict["Identifier_UPI"]][swaption_cols]
)
payer_swaption_df

,Event timestamp,Execution Timestamp,Effective Date,Expiration Date,Maturity date of the underlier,Notional amount-Leg 1,First exercise date,Fixed rate-Leg 1,Option Premium Amount,Strike Price,Unique Product Identifier,UPI FISN,UPI Underlier Name,Tenor,Option Premium Amount / $mm Notational
2762,2024-11-06 22:08:37,2024-11-06 22:08:37,2024-11-06,2026-11-06,2027-11-10,500000000.0,2026-11-06,0.038525,3122125.0,0.038525,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,2y1y,6244.25
2764,2024-11-06 22:08:37,2024-11-06 22:08:37,2024-11-06,2026-11-06,2027-11-10,500000000.0,2026-11-06,0.038525,3122125.0,0.038525,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,2y1y,6244.25
2971,2024-11-06 20:14:36,2024-11-06 20:14:36,2024-11-06,2025-01-06,2055-01-10,100000000.0,2025-01-06,0.037680,3485000.0,0.037680,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,2m30y,34850.00
2973,2024-11-06 20:14:36,2024-11-06 20:14:36,2024-11-06,2025-02-06,2055-02-10,100000000.0,2025-02-06,0.037680,3485000.0,0.037680,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,3m30y,34850.00
2980,2024-11-06 20:14:36,2024-11-06 20:14:36,2024-11-06,2025-02-06,2055-02-10,100000000.0,2025-02-06,0.037680,3485000.0,0.037680,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,3m30y,34850.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17037,2024-11-06 11:55:18,2024-11-06 11:55:18,2024-11-06,2024-12-06,2034-12-10,25000000.0,2024-12-06,0.039350,562500.0,0.039350,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00
17039,2024-11-06 11:55:18,2024-11-06 11:55:18,2024-11-06,2024-12-06,2034-12-10,25000000.0,2024-12-06,0.039350,562500.0,0.039350,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00
17305,2024-11-06 11:55:18,2024-11-06 11:55:18,2024-11-06,2024-12-06,2034-12-10,25000000.0,2024-12-06,0.039350,562500.0,0.039350,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00
17439,2024-11-06 11:32:10,2024-11-06 11:32:10,2024-11-06,2024-12-06,2034-12-10,50000000.0,2024-12-06,0.039350,1125000.0,0.039350,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,1m10y,22500.00
